# 🐠 Reef - Pytorch FasterRCNN Infer

## A self-contained, simple, pure pytorch 🔥 Faster R-CNN implementation with `LB=0.413`

![](https://storage.googleapis.com/kaggle-competitions/kaggle/31703/logos/header.png)

#### FasterR-CNN is one of the SOTA models for Object detection.

### In this notebook we present a simple solution using a pure pytorch Faster R-CNN with pretrained weights, and finetuning it for few epochs.


Is is an adapted version of [this notebook](https://www.kaggle.com/pestipeti/pytorch-starter-fasterrcnn-inference) mentioned in [this comment](https://www.kaggle.com/c/tensorflow-great-barrier-reef/discussion/290016)

## You can find the [training notebook here](https://www.kaggle.com/julian3833/coral-reef-pytorch-starter-fasterrcnn-train)


## Details: 
- FasterRCNN from torchvision
- Use Resnet50 backbone


# Please, _DO_ upvote if you find it useful or interesting!!

&nbsp;
&nbsp;
&nbsp;
&nbsp;

---

### Changelog


|Best| Version | Description|  Weights| LB |
|---| --- | ----| --- | --- |
|| [**V5**](https://www.kaggle.com/julian3833/reef-starter-torch-fasterrcnn-infer-lb-0-293?scriptVersionId=80517131)  | 2 epochs. `detection_threshold=0.5` | [coral-reef-pytorch-starter-fasterrcnn-weights](https://www.kaggle.com/julian3833/coral-reef-pytorch-starter-fasterrcnn-weights) | `0.201`|
|| [**V6**](https://www.kaggle.com/julian3833/reef-starter-torch-fasterrcnn-infer-lb-0-293?scriptVersionId=80518365)  | `detection_threshold=0.66` | [coral-reef-pytorch-starter-fasterrcnn-weights](https://www.kaggle.com/julian3833/coral-reef-pytorch-starter-fasterrcnn-weights) |`0.243`|
|| [**V7**](https://www.kaggle.com/julian3833/reef-starter-torch-fasterrcnn-infer-lb-0-293?scriptVersionId=80520095)  | `detection_threshold=0.85` |[coral-reef-pytorch-starter-fasterrcnn-weights](https://www.kaggle.com/julian3833/coral-reef-pytorch-starter-fasterrcnn-weights) | `0.292`|
|| [**V8**](https://www.kaggle.com/julian3833/reef-starter-torch-fasterrcnn-infer-lb-0-293?scriptVersionId=80534807)  | `detection_threshold=0.9` |  [coral-reef-pytorch-starter-fasterrcnn-weights](https://www.kaggle.com/julian3833/coral-reef-pytorch-starter-fasterrcnn-weights)|`0.293`|
|| [**V10**](https://www.kaggle.com/julian3833/reef-starter-torch-fasterrcnn-infer-lb-0-293)  | V8 + 4 epochs | [reef-starter-torch-fasterrcnn-4e](https://www.kaggle.com/julian3833/reef-starter-torch-fasterrcnn-4e) | `0.361`|
|| [**V11**](https://www.kaggle.com/julian3833/reef-starter-torch-fasterrcnn-infer-lb-0-361?scriptVersionId=80610091)  | Train 8 epochs with validation. Pick best epoch (6th epoch ) | [reef-starter-torch-fasterrcnn-8e](https://www.kaggle.com/julian3833/reef-starter-torch-fasterrcnn-8e) |`0.343`|
|| [**V12**](https://www.kaggle.com/julian3833/reef-starter-torch-fasterrcnn-infer-lb-0-361?scriptVersionId=80622626)  | V11 + `detection_threshold=0.66` | [reef-starter-torch-fasterrcnn-8e](https://www.kaggle.com/julian3833/reef-starter-torch-fasterrcnn-8e) |  `0.369`|
|_Best_| [**V13**](https://www.kaggle.com/julian3833/reef-starter-torch-fasterrcnn-infer-lb-0-413?scriptVersionId=80624811)  |Best of 12 epochs, lower LR (epoch=5th) + `detection_threshold=0.66` | [reef-starter-torch-fasterrcnn-12e](https://www.kaggle.com/julian3833/reef-starter-torch-fasterrcnn-12e) |  `0.413` |
|| [**V14**](https://www.kaggle.com/julian3833/reef-starter-torch-fasterrcnn-infer-lb-0-293)  | Best of 12 epochs. Split 90-10. Augmentations.  (epoch=8th) + `detection_threshold=0.66` | [reef-starter-torch-fasterrcnn-12e-v2](https://www.kaggle.com/julian3833/reef-starter-torch-fasterrcnn-12e-v2) |  `??` |


---

# Imports

In [1]:
import numpy as np
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

WEIGHTS_FILE = "../input/reef-starter-torch-fasterrcnn-12e-v2/fasterrcnn_resnet50_fpn-e7.bin"

# Model

In [2]:
def get_model():
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)
    num_classes = 2  # 1 class (starfish) + background

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # Load the trained weights
    model.load_state_dict(torch.load(WEIGHTS_FILE))
    model.eval()

    model = model.to(device)
    return model

model = get_model()

# Predict functions

In [3]:
detection_threshold = 0.66

In [4]:
def format_prediction_string(boxes, scores):
    # Format as specified in the evaluation page
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.2f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))

    return " ".join(pred_strings)


def predict(model, pixel_array):
    # Predictions for a single image
    
    # Apply all the transformations that are required
    pixel_array = pixel_array.astype(np.float32) / 255.
    tensor_img = ToTensorV2(p=1.0)(image=pixel_array)['image'].unsqueeze(0)
    
    # Get predictions
    with torch.no_grad():
        outputs = model(tensor_img.to(device))[0]
    
    # Move predictions to cpu and numpy
    boxes = outputs['boxes'].data.cpu().numpy()
    scores = outputs['scores'].data.cpu().numpy()
    
    # Filter predictions with low score
    boxes = boxes[scores >= detection_threshold].astype(np.int32)
    scores = scores[scores >= detection_threshold]
    
    # Go back from x_min, y_min, x_max, y_max to x_min, y_min, w, h
    boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
    boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
  
    # Format results as requested in the Evaluation tab
    return format_prediction_string(boxes, scores)

# Submit

## See: [Great Barrier Reef API Tutorial](https://www.kaggle.com/sohier/great-barrier-reef-api-tutorial)

In [5]:
import greatbarrierreef
env = greatbarrierreef.make_env()
iter_test = env.iter_test() 

for (pixel_array, df_pred) in iter_test:  # iterate through all test set images
    df_pred['annotations'] = predict(model, pixel_array)
    env.predict(df_pred)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


# Please, _DO_ upvote if you find it useful or interesting!!